# Chapter 4 - LDA & QDA

### Linear Discriminant Analysis and Quadratic Discriminant Analysis

This notebook presents the Python implementation of exercises from the book *"Introduction to Statistical Learning"* (a.k.a. ISLR) by Gareth James et al.

---

# Lab Exercises

## The Stock Market Data

**Dataset**: Smarket.csv; Daily percentage returns for the S&P 500 stock index between 2001 and 2005.

    
**Variables**: 1250 observations on the following 9 variables.

* Year: The year that the observation was recorded
* Lag1: Percentage return for previous day
* Lag2: Percentage return for 2 days previous
* Lag3: Percentage return for 3 days previous
* Lag4: Percentage return for 4 days previous
* Lag5: Percentage return for 5 days previous
* Volume: Volume of shares traded (number of daily shares traded in billions)
* Today: Percentage return for today
* Direction: A factor with levels Down and Up indicating whether the market had a positive or negative return on a given day

In [2]:
# import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn.linear_model as skl_lm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, precision_score
from sklearn import preprocessing
from sklearn import neighbors

import statsmodels.api as sm
import statsmodels.formula.api as smf

sns.set_style('whitegrid')
sns.set_palette('RdBu')

%matplotlib inline

C:\Users\munky\Anaconda\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [14]:
# Import data
smarket = pd.read_csv('data/ISLR/Smarket.csv', usecols=range(1,10), index_col=0, parse_dates=True)

## Linear Discriminant Analysis

In [15]:
X_train = smarket[:'2004'][['Lag1','Lag2']]
y_train = smarket[:'2004']['Direction']

X_test = smarket['2005':][['Lag1','Lag2']]
y_test = smarket['2005':]['Direction']

lda = LinearDiscriminantAnalysis()
pred = lda.fit(X_train, y_train).predict(X_test)

In [56]:
prior_probs = pd.DataFrame(lda.priors_, index=['Down', 'Up'], columns=['Probability'])
print('Prior probabilities of groups:', prior_probs.round(3), sep='\n\n')

Prior probabilities of groups:

      Probability
Down        0.492
Up          0.508


49.2% of training observations correspond to days during which the market went down.

In [55]:
group_means = pd.DataFrame(lda.means_, index=['Down', 'Up'], columns=['Lag1', 'Lag2'])
print('Group Means:', group_means.round(3), sep='\n\n')

Group Means:

       Lag1   Lag2
Down  0.043  0.034
Up   -0.040 -0.031


Group means are the average of each predictor within each class. These suggest that there is a tendency for the previous 2 days' returns to be negative on days when the market increases, and a tendency for the previous day's returns to be positive on days when the market declines.

In [58]:
coefs = pd.DataFrame(lda.coef_, index=['LD1'], columns=['Lag1', 'Lag2'])
print('Coefficients of linear discriminants:', coefs.round(3), sep='\n\n')

Coefficients of linear discriminants:

      Lag1   Lag2
LD1 -0.055 -0.044


These coefficients provide the linear combination of Lag1 and Lag2 that are used to form the LDA decision rule. For instance, if -0.055 x Lag1 - 0.044 x Lag2 is large, then the LDA classifier will predict a market increase, else a market decrease. 

In [64]:
# Create a list of LDA predictions about market movement on test set
pred = lda.predict(X_test)

# Assign posterior probabilities for each of the prediction
pred_p = lda.predict_proba(X_test)

In [88]:
# Display the confusion matrix
print('Confusion Matrix:')
cm = pd.DataFrame(confusion_matrix(y_test, pred).T, index=y_test.unique(), columns=y_test.unique())
cm.rename_axis('Pred').rename_axis("True", axis=1)

Confusion Matrix:


True,Down,Up
Pred,,
Down,35,35
Up,76,106


In [90]:
print(classification_report(y_test, pred, digits=3))

             precision    recall  f1-score   support

       Down      0.500     0.315     0.387       111
         Up      0.582     0.752     0.656       141

avg / total      0.546     0.560     0.538       252



Applying a 50% threshhold to the posterior probs allows us to recreate the predictions contained in our test prediction results.

In [102]:
np.unique(pred_p[:,1]>0.5, return_counts=True)

(array([False,  True], dtype=bool), array([ 70, 182], dtype=int64))

Note that the posterior prob output by the model corresponds to the prob that the market will **decline**.

Now suppose we wanted to use a posterior prob threshold other than 50%, say 90%. For instance, we may wish to predict a market decline only if we are very sure that the market will indeed decline on that day.

In [92]:
np.unique(pred_p[:,1]>0.9, return_counts=True)

(array([False], dtype=bool), array([252], dtype=int64))

No True values are returned; i.e., no days in 2005 meet this threshhold.

## Quadratic Discriminant Analysis

Implementing QDA is very similar to LDA.

In [103]:
X_train = smarket[:'2004'][['Lag1','Lag2']]
y_train = smarket[:'2004']['Direction']

X_test = smarket['2005':][['Lag1','Lag2']]
y_test = smarket['2005':]['Direction']

qda = QuadraticDiscriminantAnalysis()
pred = qda.fit(X_train, y_train).predict(X_test)

In [104]:
prior_probs = pd.DataFrame(qda.priors_, index=['Down', 'Up'], columns=['Probability'])
print('Prior probabilities of groups:', prior_probs.round(3), sep='\n\n')

Prior probabilities of groups:

      Probability
Down        0.492
Up          0.508


In [105]:
group_means = pd.DataFrame(qda.means_, index=['Down', 'Up'], columns=['Lag1', 'Lag2'])
print('Group Means:', group_means.round(3), sep='\n\n')

Group Means:

       Lag1   Lag2
Down  0.043  0.034
Up   -0.040 -0.031


In [107]:
# Create a list of LDA predictions about market movement on test set
pred = qda.predict(X_test)

# Assign posterior probabilities for each of the prediction
pred_p = qda.predict_proba(X_test)

In [108]:
# Display the confusion matrix
print('Confusion Matrix:')
cm = pd.DataFrame(confusion_matrix(y_test, pred).T, index=y_test.unique(), columns=y_test.unique())
cm.rename_axis('Pred').rename_axis("True", axis=1)

Confusion Matrix:


True,Down,Up
Pred,,
Down,30,20
Up,81,121


In [109]:
print(classification_report(y_test, pred, digits=3))

             precision    recall  f1-score   support

       Down      0.600     0.270     0.373       111
         Up      0.599     0.858     0.706       141

avg / total      0.599     0.599     0.559       252



Interestingly, the QDA predictions are accurate almost 60% of the time, even though the model was tested on unobserved data. This level of accuracy is quite impressive for stock market data, which is generally difficult to model accurately. 

In [115]:
# applying a posterior probability threshhold of 50%
np.unique(pred_p[:,1]>0.5, return_counts=True)

(array([False,  True], dtype=bool), array([ 50, 202], dtype=int64))

In [114]:
# applying a posterior probability threshhold of 90%
np.unique(pred_p[:,1]>0.9, return_counts=True)

(array([False], dtype=bool), array([252], dtype=int64))

---
### References:

ISLR-python: http://nbviewer.jupyter.org/github/JWarmenhoven/ISL-python/blob/master/Notebooks/Chapter%203.ipynb